In [40]:
import numpy as np
import random

In [126]:
#definições gerais
POPULATION_SIZE = 100
NUM_STRAT=10
MUTATION_RATE = 0.05
GENERATIONS = 25
BITS_PER_ROOT = 14 # Bits por número real (sinal + inteiro + fracionario + flag) = 1 + 4 + 6 + 1
NUM_ROOTS = 4 #queremos resolver um polinomio de grau 4 entao sao 4 raizes
TOTAL_BITS = BITS_PER_ROOT * NUM_ROOTS # total de bits por individuo (cromossomo)
ROOT_POOL = [3,2.5,1.5,-2]


In [130]:
#função polinomial da questão
def polynomial(x):
  return x**4 -5*x**3 + 1.75*x**2 + 20.25*x - 22.5

# Função para decodificar o valor binário do cromossomo
# Converte uma string binária em um valor de raiz e uma flag se a raiz faz parte da solucao ou nao
def decode(binary):
  sign = -1 if binary[0] == '1' else 1
  integer = int(binary[1:5],2) -8 # 4 bits da parte inteira representa de -8 a 7
  fraction = int(binary[5:13], 2) / 16 #8 bits da parte fracionaria (2^8 = 256 niveis)
  flag = int(binary[13],2) #1 bit para indicar a flag se a raiz faz parte da solucao ou nao
  root_value = sign * (integer + fraction)

  if root_value in ROOT_POOL:
    flag = '1'
  else:
    flag = '0'

  return root_value, flag

#função que indica a adaptalidade do cromossomo em questão
#pega um individuo composto de uma sequencia de 44 bits e decodificamos as 4 raizes
#f(x) é calculado para cada raiz e somamos os valores absolutos dos erros
#o sinal negativo serve para maximizar os valores, o quanto menor o erro, maior a adaptabilidade
def fitness(individual):
  roots = []
  updated_individual = ''

  for i in range(0, TOTAL_BITS, BITS_PER_ROOT):
      root_value, flag = decode(individual[i:i+BITS_PER_ROOT])
      roots.append(root_value)

      # Check for repeated roots
      if root_value in roots[:-1]:
          # Mutate the root if it is repeated
          mutated_root = mutate_single_root(individual[i:i+BITS_PER_ROOT-1])
          mutated_root += flag
          updated_individual += mutated_root
      else:
          # Update the individual if the flag was updated
          updated_individual += individual[i:i+BITS_PER_ROOT-1] + flag

  return -sum(abs(polynomial(r)) for r in roots), updated_individual

#crossover
#é escolhido um ponto aleatório de corte na string de bits de cada raiz
def crossover(parent1, parent2):
    child1 = ''
    child2 = ''
    has_repeated_root1, index1 = has_repeated_root(parent1)
    has_repeated_root2, index2 = has_repeated_root(parent2)
    for i in range(0, TOTAL_BITS, BITS_PER_ROOT):
        root1 = parent1[i:i+BITS_PER_ROOT]
        root2 = parent2[i:i+BITS_PER_ROOT]

        flag1 = root1[-1]
        flag2 = root2[-1]

        point = random.randint(1, BITS_PER_ROOT - 1)

        #prioriza as raizes repetidas para o crossover
        if has_repeated_root1 and i == index1:
            child1 += root1[:point] + root2[point:]
            child2 += root2[:point] + root1[point:]
        elif has_repeated_root2 and i == index2:
            child1 += root2[:point] + root1[point:]
            child2 += root1[:point] + root2[point:]
        else:
            if flag1 == '0' and flag2 == '0':
                child1 += root1[:point] + root2[point:]
                child2 += root2[:point] + root1[point:]
            elif flag1 == '0' and flag2 == '1' :
                child1 += root1[:point] + root2[point:]
                child2 += root2
            elif flag1 == '1' and flag2 == '0':
                child1 += root1
                child2 += root2[:point] + root1[point:]
            else:
                child1 += root1
                child2 += root2
    return child1, child2


def mutate_single_root(root):
    mutated = ''
    for bit in root[:-1]:  # Exclude the flag bit from mutation
        if random.random() < 0.3:
            mutated += '1' if bit == '0' else '0'
        else:
            mutated += bit
    mutated += root[-1]
    return mutated

def has_repeated_root(individual):
    roots = []
    for i in range(0, TOTAL_BITS, BITS_PER_ROOT):
        root_value, _ = decode(individual[i:i+BITS_PER_ROOT])
        if root_value in roots:
            return True, i  # Return True immediately if a repeated root is found
        roots.append(root_value)
    return False, -1  # Return False if no repeated roots are found

#nao realiza mutacao na raiz se flag for 1, caso flag seja 0, realiza mutacao na raiz, caso a raiz seja repetida, realiza mutacao na raiz
#mutação uniforme
def mutation(individual):
    mutated = ''
    has_repeat, index = has_repeated_root(individual)

    for i in range(0, TOTAL_BITS, BITS_PER_ROOT):
        root = individual[i:i+BITS_PER_ROOT]
        flag = root[-1]  # The last bit is the flag

        if flag == '0' or (i == index and has_repeat):  # Only mutate if the flag is '0'
            for bit in root[:-1]:  # Exclude the flag bit from mutation
                if random.random() < MUTATION_RATE:
                    mutated += '1' if bit == '0' else '0'
                else:
                    mutated += bit
            mutated += flag  # Append the unchanged flag
        else:
            mutated += root  # If flag is '1', keep the root unchanged
    return mutated

#(k=3)individuos sao escolhidos aleatoriamente e o melhor indivduo é escolhido
def tournament_selection(population, scores, k=3):
  #zip(population,scores) junta as duas listas em uma lista de duplas
  selected = random.sample(list(zip(population, scores)),k)
  return max(selected, key=lambda x: x[1])[0] #pega o primeiro elemento da lista {population,scores} (com o maior valor fitness (selected[1]))

#geração da população inicial com individuos aleatórios
def generate_population():
  population = []
  for i in range(POPULATION_SIZE):
    individual = ''
    for j in range(0, TOTAL_BITS, BITS_PER_ROOT):
        for k in range(BITS_PER_ROOT-1):
            individual += str(random.randint(0,1))
        individual += str(0) #flag para indicar que a raiz nao faz parte da solucao
    population.append(individual)
  return population

#dessa vez usamos seleção elitista, selecionamos os melhores individuos para a nova população e preenchemos o restante com descendentes
def genetic_algorithm():
    population = generate_population()
    for _ in range(GENERATIONS):
        scores = []
        updated_population = []  # população com a flag atualizada
        for individual in population:  # avalia a população da geração atual
            score, updated_individual = fitness(individual)
            scores.append(score)
            updated_population.append(updated_individual)

        # Sort population by fitness scores in descending order
        sorted_population = [x for _, x in sorted(zip(scores, updated_population), reverse=True)]

        # Elitist selection: carry over the top individuals
        new_population = sorted_population[:POPULATION_SIZE // 2]#seleciona os melhores individuos para a nova população

        # Fill the rest of the new population with offspring
        while len(new_population) < POPULATION_SIZE:
            parent1 = random.choice(sorted_population)
            parent2 = random.choice(sorted_population)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutation(child1))#adiciona o filho 1
            if len(new_population) < POPULATION_SIZE:#se ainda nao atingiu o tamanho da população, adiciona o filho 2
                new_population.append(mutation(child2))

        population = new_population

    #max() pegar o valor maximo na lista de individuos
    #key=lambda ind: fitness(ind)[0] chama a função fitness para cada individuo da população
    #'key=' é uma função lambda que chama uma função fitness para cada individuo da população
    #'ind' é o individuo da população
    #'fitness(ind)[0]' é o valor da função fitness para o individuo 'ind', fitness retorna uma tupla com o valor da função e o individuo atualizado
    best_individual = max(population, key=lambda ind: fitness(ind)[0])  # pra cada individuo da população, a função fitness é chamada para ele
    best_roots = []
    for i in range(0, TOTAL_BITS, BITS_PER_ROOT):
        new_root, flag= decode(best_individual[i:i+BITS_PER_ROOT])
        best_roots.append(new_root)
    return best_roots

test_roots = genetic_algorithm()
print(test_roots)



[1.5, 2.5, 3.0, -2.0]


In [ ]:
def genetic_algorithm():
    population = generate_population()
    for generation in range(GENERATIONS):
        scores = []
        updated_population = []  # Population with updated flags
        repeat_flags = []  # Track individuals with repeated roots

        for individual in population:  # Evaluate the current generation's population
            score, updated_individual, has_repeats = fitness(individual)
            scores.append(score)
            updated_population.append(updated_individual)
            repeat_flags.append(has_repeats)

        # Sort population by fitness scores in descending order
        sorted_population = [x for _, x in sorted(zip(scores, updated_population), reverse=True)]
        sorted_repeats = [x for _, x in sorted(zip(scores, repeat_flags), reverse=True)]

        # Elitist selection: carry over the top individuals
        new_population = sorted_population[:POPULATION_SIZE // 2]

        # Fill the rest of the new population with offspring
        while len(new_population) < POPULATION_SIZE:
            # Prioritize individuals with repeated roots for crossover
            if any(sorted_repeats):
                repeat_indices = [i for i, x in enumerate(sorted_repeats) if x]
                parent1 = sorted_population[random.choice(repeat_indices)]
                parent2 = sorted_population[random.choice(repeat_indices)]
            else:
                parent1 = random.choice(sorted_population)
                parent2 = random.choice(sorted_population)

            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutation(child1, generation, GENERATIONS))
            if len(new_population) < POPULATION_SIZE:
                new_population.append(mutation(child2, generation, GENERATIONS))

        population = new_population

    best_individual = max(population, key=lambda ind: fitness(ind)[0])
    best_roots = []
    for i in range(0, TOTAL_BITS, BITS_PER_ROOT):
        new_root, flag= decode(best_individual[i:i+BITS_PER_ROOT])
        best_roots.append(new_root)
    return best_roots

In [ ]:
#algoritmo genetico usando seleção por torneio
def genetic_algorithm():
    population = generate_population()
    for _ in range(GENERATIONS):
        scores = []
        updated_population = [] #população com a flag atualizada
        for individual in population:  # avalia a população da geração atual
            score, updated_individual = fitness(individual)
            scores.append(score)
            updated_population.append(updated_individual)
        new_population = []
        for i in range(POPULATION_SIZE // 2):
            parent1 = tournament_selection(updated_population, scores)
            parent2 = tournament_selection(updated_population, scores)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutation(child1))
            new_population.append(mutation(child2))
        population = new_population

    best_individual = max(population, key=lambda ind: fitness(ind)[0])  # pra cada individuo da população, a função fitness é chamada para ele
    best_roots = []
    for i in range(0, TOTAL_BITS, BITS_PER_ROOT):
        best_roots.append(decode(best_individual[i:i+BITS_PER_ROOT])[0])
    return best_roots